In [7]:
#from https://github.com/langchain-ai/langchain/blob/master/cookbook/Semi_Structured_RAG.ipynb
# !pip install -- update chromadb langchain
# !pip show chromadb langchain
import os

os.environ['OPENAI_API_KEY'] = '<OPEN_AI_KEY_HERE>'

# Access the environment variable later in your code

!env

SHELL=/bin/bash
SESSION_MANAGER=local/oscar-x230:@/tmp/.ICE-unix/1128,unix/oscar-x230:/tmp/.ICE-unix/1128
WINDOWID=65011715
QT_ACCESSIBILITY=1
COLORTERM=truecolor
XDG_CONFIG_DIRS=/etc/xdg/xdg-xubuntu:/etc/xdg:/etc/xdg
XDG_SESSION_PATH=/org/freedesktop/DisplayManager/Session0
XDG_MENU_PREFIX=xfce-
CLUTTER_BACKEND=x11
LANGUAGE=en_GB:en
SSH_AUTH_SOCK=/run/user/1000/keyring/ssh
OPENAI_API_KEY=<OPEN_AI_KEY_HERE>
DESKTOP_SESSION=xubuntu
SSH_AGENT_PID=1209
XDG_SEAT=seat0
PWD=/home/oscar/Documents/scripts
XDG_SESSION_DESKTOP=xubuntu
LOGNAME=oscar
QT_QPA_PLATFORMTHEME=gtk2
XDG_SESSION_TYPE=x11
JPY_SESSION_NAME=/home/oscar/Documents/scripts/Untitled1.ipynb
GPG_AGENT_INFO=/run/user/1000/gnupg/S.gpg-agent:0:1
_=/usr/bin/env
XAUTHORITY=/home/oscar/.Xauthority
XDG_GREETER_DATA_DIR=/var/lib/lightdm-data/oscar
GDM_LANG=en_GB
HOME=/home/oscar
LANG=en_GB.UTF-8
LS_COLORS=rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:o

In [1]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
from typing import Any
import chromadb
import chromadb.config
from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


import uuid

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

In [9]:
# Get elements


raw_pdf_elements = partition_pdf(
    filename="/home/oscar/Documents/scripts/Recommendations_of_the_Taskforce_on_Nature-related_Financial_Disclosures_September_2023.pdf",
    # Unstructured first finds embedded image blocks
    extract_images_in_pdf=False,
    # Use layout model (YOLOX) to get bounding boxes (for tables) and find titles
    # Titles are any sub-section of the document
    infer_table_structure=True,
    # Post processing to aggregate text once we have the title
    chunking_strategy="by_title",
    # Chunking params to aggregate text blocks
    # Attempt to create a new chunk 3800 chars
    # Attempt to keep chunks > 2000 chars
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path="/home/oscar/Documents/scripts/outputs/",
)



Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:


# Create a dictionary to store counts of each type
category_counts = {}

for element in raw_pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
category_counts



{"<class 'unstructured.documents.elements.CompositeElement'>": 147,
 "<class 'unstructured.documents.elements.Table'>": 50}

In [11]:
class Element(BaseModel):
    type: str
    text: Any


# Categorize by type
categorized_elements = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        categorized_elements.append(Element(type="table", text=str(element)))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        categorized_elements.append(Element(type="text", text=str(element)))

# Tables
table_elements = [e for e in categorized_elements if e.type == "table"]
print(len(table_elements))

# Text
text_elements = [e for e in categorized_elements if e.type == "text"]
print(len(text_elements))

50
147


In [22]:
# Prompt
import time
prompt_text = """You are an assistant tasked with summarizing tables and text. \ 
Give a concise summary of the table or text. Table or text chunk: {element} """
prompt = ChatPromptTemplate.from_template(prompt_text)

# Summary chain
model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", api_key='<OPEN_AI_KEY_HERE>')
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser() 


In [14]:
# Apply to tables
tables = [i.text for i in table_elements]
# print(tables[0:3])
# table_summaries = summarize_chain.batch(tables[0:2], {"max_concurrency": 2})
# table_summaries

# table_list=[]
# i=0
# for table in tables: 
#     i+=1
#     table_summary = summarize_chain.invoke(table)
#     table_list.append(table_summary)
#     if i==4|i==5:
#         time.sleep(60) 
#         i=0
# # table_list
# print(len(table_list))
# print(len(tables))
 

In [15]:
# Apply to texts`1
texts = [i.text for i in text_elements]
# text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})

# text_list=[]
# i=0
# for text in texts[145:]: 
#     i+=1
#     text_summary = summarize_chain.invoke(text)
#     print(text_summary)
#     text_list.append(text_summary)
#     if i==5:
#         time.sleep(10) 
#         i=0
# len(text_list)

In [126]:
import json

with open("text_list.json", 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(text_list, f, indent=2) 

with open("table_list.json", 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(table_list, f, indent=2) 




In [12]:
import json
with open("text_list.json", 'r') as f:
    text_list = json.load(f)

with open("table_list.json", 'r') as f:
    table_list = json.load(f)


In [29]:
#Adding to vector store 

# import os
# import openai
 
# openai.api_key = os.environ["OPENAI_API_KEY"]
from langchain.vectorstores import Chroma 
# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="summaries", embedding_function=OpenAIEmbeddings())

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(text_list)
]

retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [
    Document(page_content=s, metadata={id_key: table_ids[i]})
    for i, s in enumerate(table_list)
]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))



In [30]:
#Run RAG 


from langchain_core.runnables import RunnablePassthrough

# Prompt template
template = """Answer the question based only on the following context, which can include text and tables:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# LLM
model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

# RAG pipeline
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)



In [22]:
chain.invoke("What are the key principles of the TNFD? Produce your answer in markdown")

"The key principles of the TNFD are as follows:\n\n1. Globally inclusive: The framework and approach should be relevant and accessible worldwide, across emerging and developed markets.\n\n2. Market usability: The framework should be directly usable and valuable to market participants, including corporates and financial institutions, as well as policy and other actors.\n\n3. Science-based: The framework should follow a scientifically anchored approach, incorporating well-established and emerging scientific evidence, and converging towards other existing science-based initiatives.\n\n4. Nature-related risks: The framework should embrace nature-related risks, including immediate and material financial risks, as well as nature dependencies and impacts and their related organizational and societal risks.\n\n5. Purpose driven: The framework should actively reduce risks and increase nature-positive action by using the minimum required level of granularity to ensure the achievement of the TNFD

In [23]:
chain.invoke("Explain the TNFD to a 5 year old")

'The TNFD, which stands for Taskforce on Nature-related Financial Disclosures, is a group of people who are working together to help organizations understand and talk about the impact they have on nature. They want to make sure that companies and other groups are aware of how their actions can affect the environment and the plants and animals that live in it. The TNFD is creating a set of guidelines and recommendations to help organizations report on their impact on nature and take steps to protect it. They are also working with scientists and experts to make sure their recommendations are based on the best available information.'

In [31]:
chain.invoke("What are the next phase priorities for the TNFD?")

'The next phase priorities for the TNFD are outlined in Table 5.'

In [35]:
table_elements[3]

Element(type='table', text='Category Description Acute risks Occurrenceofshortterm,specificeventsthatchangethestateofnature.Forexample,oilspills, forestfiresorpestsaffectingaharvest. Chronic risks Gradual changes to the state of nature. For example, pollution stemming from pesticide use or climate change.')

In [37]:
table_list[3]

'The table describes two categories of risks: acute risks, which are short-term events that change the state of nature, such as oil spills or forest fires, and chronic risks, which are gradual changes to the state of nature, such as pollution from pesticide use or climate change.'

In [39]:
retriever 

MultiVectorRetriever(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7face384a400>, docstore=<langchain.storage.in_memory.InMemoryBaseStore object at 0x7facea126880>)